# Dashboard - SIR Model

## Name: Rohit Dev

In [ ]:
# Import libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import missingno as msno
import plotly.graph_objects as go

from datetime import datetime

In [ ]:
%matplotlib inline
sns.set(style='darkgrid')
mpl.rcParams['figure.figsize'] = (16,9)
pd.set_option('display.max_rows', 500)

In [ ]:
# SIR Model is being for shown 3 countries
country_list = ['Germany', 'India', 'Spain']

In [ ]:
# Import raw data
raw_data=pd.read_csv('../data/raw/owid-covid-data.csv')

In [ ]:
# display raw data
raw_data.head()

In [ ]:
# raw_data columns list
raw_data.columns

In [ ]:
# Sorting of required data from raw data  
data_sort = raw_data[['date', 'location','total_cases', 'population']]

In [ ]:
# Changing column names
data_sort = data_sort.rename(columns={'location':'country', 'total_cases':'infected'})

In [ ]:
data_sort.head()

In [ ]:
# Changing date format
date_idx = [datetime.strptime(each, '%Y-%m-%d') for each in data_sort.date]
data_sort['date'] = date_idx

In [ ]:
df_processed = pd.DataFrame(data_sort['date'])

In [ ]:
df_processed = pd.DataFrame(df_processed['date'].unique())

In [ ]:
df_processed.rename(columns={0:'date'}, inplace=True)

In [ ]:
df_processed.sort_values(by='date', inplace=True)

In [ ]:
df_processed.set_index(['date'], inplace=True)

In [ ]:
data_sort.set_index(['date'], inplace=True)

In [ ]:
# Create a new processed data frame with country infected cases along with population size 
for each in country_list:
    df_processed[each+'_infected'] = pd.DataFrame(data_sort[data_sort['country'] == each]['infected'])
    df_processed[each+'_population'] = pd.DataFrame(data_sort[data_sort['country'] == each]['population'])

In [ ]:
df_processed.head()

In [ ]:
# Fill 0 value in empty cells
df_population = df_processed.fillna(0)

In [ ]:
country_population_list=np.zeros(len(country_list))
for pos, each in enumerate(country_list):
    country_population_list[pos] = max(df_population[each+'_population'])

In [ ]:
# Import for SIR Model 
from scipy import optimize
from scipy import integrate

In [ ]:
ydata_Germany = np.array(df_processed.Germany_infected[60:]) # 60 entries have been taken
ydata_Germany = ydata_Germany.astype(int)
t_Germany=np.arange(len(ydata_Germany))

In [ ]:
df_processed['Germany_population']

In [ ]:
# Parameters

#N0 = country_population_list[1] # Population size
N0 = 1000000
beta = 0.4
gamma = 0.1  # 1(gamma) Person can infect 4(beta) people

I0 = ydata_Germany[0]   # Initial infected size
#S0 = N0-I0  # Population prone to infection
S0 = N0-I0
R0 = 0  # Initial recovered size

In [ ]:
I0

In [ ]:
# SIR Model Function

def SIR_model(SIR, t, beta, gamma):
    ''' Simple SIR model
        S: susceptible population
        I: infected people
        R: recovered people
        beta:
        
        overall condition is that the sum of changes (differences) sum up to 0
        dS+dI+dR=0
        S+I+R = N (constant size of population)
    '''
    
    S,I,R = SIR
    dS_dt = -beta*S*I/N0
    dI_dt = beta*S*I/N0-gamma*I
    dR_dt = gamma*I
    return ([dS_dt, dI_dt, dR_dt])

In [ ]:
def fit_odeint(x, beta, gamma):
    return integrate.odeint(SIR_model, (S0, I0, R0), t_Germany, args=(beta, gamma))[:,1] # we only would like to get dI

In [ ]:
popt=[0.4, 0.1]
fit_odeint(t_Germany, *popt)

In [ ]:
popt_Germany, pcov_Germany = optimize.curve_fit(fit_odeint, t_Germany, ydata_Germany)
perr_Germany = np.sqrt(np.diag(pcov_Germany))

In [ ]:
print('standard deviation error: ', str(perr_Germany), 'start infect:', ydata_Germany[0])
print('Optimal parameters: beta = ', popt_Germany[0], 'and gamma = ', popt_Germany[1])

In [ ]:
fitted_Germany = fit_odeint(t_Germany, *popt_Germany)

In [ ]:
plt.semilogy(t_Germany, ydata_Germany, 'o',color='g')
plt.semilogy(t_Germany, fitted_Germany, color='g')
plt.title('Fit of SIR model for Germany cases')
plt.ylabel('Population infected')
plt.xlabel('Days')
plt.show()
print('Optimal parameters: beta = ', popt_Germany[0], 'and gamma = ', popt_Germany[1])
print('Basic Reproduction Number R0: ',popt_Germany[0]/popt_Germany[1] )

# India

In [ ]:
ydata_India = np.array(df_processed.India_infected[70:])
#ydata_India = ydata_India.astype(int)
t_India=np.arange(len(ydata_India))

In [ ]:
# Parameters

#N0 = country_population_list[1] # Population size
N0 = 10000000
beta = 0.4
gamma = 0.1  # 1(gamma) Person can infect 4(beta) people

I0 = ydata_India[0]   # Initial infected size
#S0 = N0-I0  # Population prone to infection
S0 = N0-I0
R0 = 0  # Initial recovered size

In [ ]:
I0

In [ ]:
def fit_odeint(x, beta, gamma):
    return integrate.odeint(SIR_model, (S0, I0, R0), t_India, args=(beta, gamma))[:,1] # we only would like to get dI

In [ ]:
popt=[0.4, 0.1]
fit_odeint(t_India, *popt)

In [ ]:
popt_India, pcov_India = optimize.curve_fit(fit_odeint, t_India, ydata_India)
perr_India = np.sqrt(np.diag(pcov_India))

In [ ]:
print('standard deviation error: ', str(perr_India), 'start infect:', ydata_India[0])
print('Optimal parameters: beta = ', popt_India[0], 'and gamma = ', popt_India[1])

In [ ]:
fitted_India = fit_odeint(t_India, *popt_India)

In [ ]:
plt.semilogy(t_India, ydata_India, 'o',color='y')
plt.semilogy(t_India, fitted_India, color='y')
plt.title('Fit of SIR model for India cases')
plt.ylabel('Population infected')
plt.xlabel('Days')
plt.show()
print('Optimal parameters: beta = ', popt_India[0], 'and gamma = ', popt_India[1])
print('Basic Reproduction Number R0: ',popt_India[0]/popt_India[1] )

# Spain

In [ ]:
ydata_Spain = np.array(df_processed.Spain_infected[60:])
t_Spain=np.arange(len(ydata_Spain))

In [ ]:
# Parameters

#N0 = country_population_list[1] # Population size
N0 = 1000000
beta = 0.4
gamma = 0.1  # 1(gamma) Person can infect 4(beta) people

I0 = ydata_Spain[0]   # Initial infected size
#S0 = N0-I0  # Population prone to infection
S0 = N0-I0
R0 = 0  # Initial recovered size

In [ ]:
I0

In [ ]:
def fit_odeint(x, beta, gamma):
    return integrate.odeint(SIR_model, (S0, I0, R0), t_Spain, args=(beta, gamma))[:,1] # we only would like to get dI

In [ ]:
popt=[0.4, 0.1]
fit_odeint(t_Spain, *popt)

In [ ]:
popt_Spain, pcov_Spain = optimize.curve_fit(fit_odeint, t_Spain, ydata_Spain)
perr_Spain = np.sqrt(np.diag(pcov_Spain))

In [ ]:
print('standard deviation error: ', str(perr_Spain), 'start infect:', ydata_Spain[0])
print('Optimal parameters: beta = ', popt_Spain[0], 'and gamma = ', popt_Spain[1])

In [ ]:
fitted_Spain = fit_odeint(t_Spain, *popt_Spain)

In [ ]:
plt.semilogy(t_Spain, ydata_Spain, 'o',color='r')
plt.semilogy(t_Spain, fitted_Spain, color='r')
plt.title('Fit of SIR model for Spain cases')
plt.ylabel('Population infected')
plt.xlabel('Days')
plt.show()
print('Optimal parameters: beta = ', popt_Spain[0], 'and gamma = ', popt_Spain[1])
print('Basic Reproduction Number R0: ',popt_Spain[0]/popt_Spain[1] )

## Combining all SIR Data from 3 countries in one data frame

In [ ]:
SIR=pd.DataFrame(t_Germany)

In [ ]:
SIR.rename(columns={0:'date'}, inplace=True)

In [ ]:
SIR['Germany_infected'], SIR['Germany_infected_fitted'] =pd.DataFrame(ydata_Germany), pd.DataFrame(fitted_Germany)

In [ ]:
SIR.head()

In [ ]:
SIR['India_infected'], SIR['India_infected_fitted'] =pd.DataFrame(ydata_India), pd.DataFrame(fitted_India)

In [ ]:
SIR.tail()

In [ ]:
SIR['Spain_infected'], SIR['Spain_infected_fitted'] =pd.DataFrame(ydata_Spain), pd.DataFrame(fitted_Spain)

In [ ]:
SIR.head()

# Dashboard Preparation

In [ ]:
# country drop-down list
country_select_list = []
for each in country_list:
    select = {'label':each, 'value':each+'_infected'}
    country_select_list.append(select)

In [ ]:
country_select_list

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html

fig = go.Figure()
app = dash.Dash()

app.layout = html.Div([
    
    dcc.Markdown('''
    #  Enterprise Data Science - (COVID-19 Data SIR Model)
    '''),



    html.Label(children='Multi-Select Country for visualization'),
    dcc.Dropdown(id='country_drop_down', options=country_select_list, multi=False, value=country_select_list[0]['value'], placeholder='Select the country here...'),
    dcc.Graph(figure=fig, id='main_window_slope') # Output
])

In [ ]:
from dash.dependencies import Input, Output

@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value')])
def update_function(country_drop_down):
    
    traces = []
    
    traces.append(dict(x=SIR.index,
                       y=SIR[country_drop_down],
                         mode='markers',
                         opacity=0.9,
                         line_width=2,
                         marker_size=4,
                         name=country_drop_down))
    traces.append(dict(x=SIR.index,
                       y=SIR[country_drop_down+str('_fitted')],
                         mode='lines',
                         opacity=0.9,
                         line_width=2,
                         marker_size=4,
                         name=country_drop_down+str('_fitted')))
           

    
    return {
            'data': traces,
            'layout': dict(
                width=1280,
                height=720,
                
                xaxis={'title': 'Days',
                        'tickangle':-45,
                        'nticks':20,
                       'tickfont':dict(size=14, color="#7f7f7f"),
                      },
                
                yaxis={'title': 'Infected Population',
                    'type':"linear",
                       'range':'[1.1,5.5]'}
            )}

    

In [ ]:
app.run_server(debug=True, use_reloader=False)

# End